In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
class Sensor:
    """
    one line following sensor, for the purpose of this implementation we are using 
    basic sensor that will return the square of distance from this sensor to the line
    """
    def __self__(self,model=None
                 ,add_noise=0.05 #add 5% noise to the simulated measurments
                ):
        
        self.model=model
        self.noise=add_noise
        
    def value(self,dist):
        if model:
            return self.model*(1+(1-np.random.random())*2*self.noise)
        else:
            return dist**2*(1+(1-np.random.random())*2*self.noise)
        
    def fit(self):
        """
        for fitting, for this I would need data arrrrgh
        """
        pass

In [ ]:
class Line_array:
    def __init__(self,start_X,start_Y,length,distance,start_orientation=0):
        """
        start_X, start_Y is the loc of the center of the array
        start_orientation, the inital starting orientation
        """
        
        self.